In [10]:
from tree_sitter_language_pack import get_binding, get_language, get_parser

python_binding = get_binding('python')  # this is an int pointing to the C binding
python_lang = get_language('python')  # this is an instance of tree_sitter.Language
python_parser = get_parser('python')  # this is an instance of tree_sitter.Parser

In [32]:
python_node = python_parser.parse(bytes("""
class Node:
    def __init__(self):
        self.x = 10
    def return_type(self):
        return self.x
""", "utf8"))

python_node.root_node.type

'module'

In [33]:
root_node = python_node.root_node
print(root_node.type)
print(root_node.text)

module
b'class Node:\n    def __init__(self):\n        self.x = 10\n    def return_type(self):\n        return self.x\n'


In [34]:
# get functions from root node 
root_node.children

[<Node type=class_definition, start_point=(1, 0), end_point=(5, 21)>]

In [35]:
# ge

1

In [11]:
javascript_binding = get_binding('javascript')  # this is an int pointing to the C binding
javascript_lang = get_language('javascript')  # this is an instance of tree_sitter.Language
javascript_parser = get_parser('javascript')  # this is an instance of tree_sitter.Parser

In [13]:
js_node = javascript_parser.parse(bytes("""
function hello() {
    return "Hello, world!";
}
""", "utf8"))

In [14]:
js_node.root_node.type

'program'

In [36]:
def extract_nodes(tree, source_code):
    def get_node_text(node):
        return source_code[node.start_byte:node.end_byte].decode('utf-8')

    def get_context(node):
        return {
            "module": "settings",
            "file_path": "/Users/abcom/Desktop/github/reviewturtl/reviewturtl/settings.py",
            "file_name": "settings.py",
            "struct_name": "Settings",
            "snippet": get_node_text(node)
        }

    root_node = tree.root_node
    class_node = None
    results = []

    # Traverse the tree to find classes and methods
    def traverse(node):
        nonlocal class_node
        if node.type == 'class_definition':
            class_node = node
            class_info = {
                "name": get_node_text(node.child_by_field_name('name')),
                "signature": get_node_text(node),
                "code_type": "Class",
                "docstring": None,
                "line": node.start_point[0] + 1,
                "line_from": node.start_point[0] + 1,
                "line_to": node.end_point[0] + 1,
                "context": get_context(node)
            }
            results.append(class_info)
        elif node.type == 'function_definition' and class_node:
            method_info = {
                "name": get_node_text(node.child_by_field_name('name')),
                "signature": get_node_text(node),
                "code_type": "Method",
                "docstring": None,
                "line": node.start_point[0] + 1,
                "line_from": node.start_point[0] + 1,
                "line_to": node.end_point[0] + 1,
                "context": get_context(node)
            }
            results.append(method_info)
        for child in node.children:
            traverse(child)

    traverse(root_node)
    return results

In [42]:
source_code = bytes("""
                    class Node: 
                        def __init__(self): 
                            self.x = 10 
                        def return_type(self): 
                            return self.x""","utf-8")
python_node = python_parser.parse(source_code)
extract_nodes(python_node, source_code)

[{'name': 'Node',
  'signature': 'class Node: \n                        def __init__(self): \n                            self.x = 10 \n                        def return_type(self): \n                            return self.x',
  'code_type': 'Class',
  'docstring': None,
  'line': 2,
  'line_from': 2,
  'line_to': 6,
  'context': {'module': 'settings',
   'file_path': '/Users/abcom/Desktop/github/reviewturtl/reviewturtl/settings.py',
   'file_name': 'settings.py',
   'struct_name': 'Settings',
   'snippet': 'class Node: \n                        def __init__(self): \n                            self.x = 10 \n                        def return_type(self): \n                            return self.x'}},
 {'name': '__init__',
  'signature': 'def __init__(self): \n                            self.x = 10',
  'code_type': 'Method',
  'docstring': None,
  'line': 3,
  'line_from': 3,
  'line_to': 4,
  'context': {'module': 'settings',
   'file_path': '/Users/abcom/Desktop/github/reviewturtl